<a href="https://colab.research.google.com/github/JKH-ML/python/blob/main/10_%EB%82%98%EC%9D%B4%EB%B8%8C_%EB%B2%A0%EC%9D%B4%EC%A6%88_%EB%B6%84%EB%A5%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10. 나이브 베이즈 분류기

## 개요

나이브 베이즈 분류기(Naive Bayes Classifier)는 **베이즈 정리(Bayes' Theorem)**를 기반으로 하는 **확률 기반의 분류 알고리즘**입니다.  
"나이브(naive)"라는 이름은 모든 특성(feature)들이 **서로 독립적(independent)**이라고 가정하기 때문에 붙었습니다.  
이러한 가정은 현실과는 다소 다르지만, 실제로는 매우 좋은 성능을 보이는 경우가 많습니다.

---

## 베이즈 정리

베이즈 정리는 다음과 같은 형태로 표현됩니다:

$$
P(C \mid X) = \frac{P(X \mid C) \cdot P(C)}{P(X)}
$$

- $P(C \mid X)$: 주어진 특성 $X$일 때 클래스 $C$일 확률 (사후 확률)
- $P(X \mid C)$: 클래스 $C$일 때 특성 $X$일 확률 (우도)
- $P(C)$: 클래스 $C$의 사전 확률
- $P(X)$: 특성 $X$의 전체 확률

분류 시, $P(X)$는 모든 클래스에 대해 동일하므로 생략해도 됩니다.

---

## 나이브 가정

$X = (x_1, x_2, ..., x_n)$ 이고, 이들이 서로 조건부 독립이라면:

$$
P(X \mid C) = P(x_1 \mid C) \cdot P(x_2 \mid C) \cdot \dots \cdot P(x_n \mid C)
$$

따라서 나이브 베이즈 분류기는 다음과 같이 최대 확률을 갖는 클래스를 선택합니다:

$$
\hat{C} = \arg\max_C P(C) \cdot \prod_{i=1}^{n} P(x_i \mid C)
$$

---

## 종류

- **가우시안 나이브 베이즈**: 특성이 연속적인 값을 가질 때 정규분포 가정
- **멀티노미얼 나이브 베이즈**: 텍스트 데이터처럼 카운트 기반 입력에 적합
- **베르누이 나이브 베이즈**: 이진(0 또는 1) 특성에 적합

---

## 특징

- 빠르고 구현이 간단하다
- 고차원 데이터에 강함 (ex. 텍스트 분류)
- 변수 간 독립성 가정은 현실적이지 않지만 예외적으로 잘 작동함

---

## 구현 순서 (가우시안 나이브 베이즈)

1. 클래스별 사전 확률 $P(C)$ 계산
2. 각 클래스에서 각 특성의 평균과 표준편차 계산 (정규분포 가정)
3. 각 특성값에 대해 정규분포 확률밀도 함수 계산
4. 클래스별 사후 확률 계산 후, 최대값 선택

---

## 정규분포 확률 밀도 함수

$$
P(x_i \mid C) = \frac{1}{\sqrt{2\pi\sigma_C^2}} \exp\left(-\frac{(x_i - \mu_C)^2}{2\sigma_C^2}\right)
$$

여기서 $\mu_C$는 해당 클래스의 평균, $\sigma_C$는 표준편차입니다.

---

## 실습 예제

- 이진 분류
- 2차원 연속 입력 (가우시안 나이브 베이즈)
- 시각화를 통해 결정경계 확인"


In [1]:
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB

# 1. 데이터 생성
X, y = make_classification(
    n_samples=200, n_features=2, n_informative=2,
    n_redundant=0, n_clusters_per_class=1, random_state=42
)

# 2. 클래스별 통계량 구하기
def summarize_by_class(X, y):
    summaries = {}
    for cls in np.unique(y):
        X_c = X[y == cls]
        mean = X_c.mean(axis=0)
        std = X_c.std(axis=0)
        prior = len(X_c) / len(X)
        summaries[cls] = {"mean": mean, "std": std, "prior": prior}
    return summaries

# 3. 정규분포 확률 밀도 함수
def gaussian_prob(x, mean, std):
    exponent = np.exp(-((x - mean)**2) / (2 * std**2))
    return (1 / (np.sqrt(2 * np.pi) * std)) * exponent

# 4. 예측 함수
def predict(X, summaries):
    predictions = []
    for x in X:
        probs = {}
        for cls, stats in summaries.items():
            prob = stats["prior"]
            for i in range(len(x)):
                prob *= gaussian_prob(x[i], stats["mean"][i], stats["std"][i])
            probs[cls] = prob
        predictions.append(max(probs, key=probs.get))
    return np.array(predictions)

# 5. 직접 구현 예측
summaries = summarize_by_class(X, y)
y_pred_manual = predict(X, summaries)

# 6. Sklearn 예측
model = GaussianNB()
model.fit(X, y)
y_pred_sklearn = model.predict(X)

# 7. 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers',
                         marker=dict(color=y_pred_manual, symbol='circle', size=10),
                         name="Naive Bayes (manual)"))

fig.update_layout(title="Naive Bayes 분류기 - NumPy 구현",
                  xaxis_title="X1", yaxis_title="X2", width=700, height=500)
fig.show()
